### **Creación de variables agregadas a partir del dataset del dataset [Cost of Living](https://www.kaggle.com/datasets/mvieira101/global-cost-of-living/code)**

Este notebook implementa la creación de **7 índices calculados** sobre el dataset `cost-of-living-clean.csv`.

La idea general es que variables desagregadas sobre el coste de frutas o verduras por separada aportan poco valor analítico. 

En cambio el cálculo de variables agregadas relacionadas con coste de la cesta de compra o precio de la vivienda sí lo hacen. 

#### **Pasos a seguir y descripción de variables agregadas**

1. **Setup y carga de datos**: Importar librerías y cargar `cost-of-living-clean.csv`
2. **Exploración inicial**: Verificar columnas disponibles y tipos de datos
3. **Variables agregada 1 — `nomad_housing_cost`**: promedio alquiler 1br (centro + afueras) / 2
4. **Variables agregada 2 — `basic_basket_index`**: promedio productos básicos supermercado
5. **Variables agregada 3 — `daily_meal_cost`**: cappuccino + comida restaurante económico
6. **Variables agregada 4 — `monthly_nomad_cost`**: coste mensual total (vivienda + comida + internet + utilities + transporte)
7. **Variables agregada 5 — `local_purchasing_power`**: salario / coste mensual nómada
8. **Variables agregada 6 — `cappuccino_index`**: normalización del precio cappuccino
9. **Variables agregada 7 — `housing_salary_ratio`**: (alquiler / salario) × 100
10. **Creación del nuevo CSV con la variables agregadas**: creación de nuevas columnas y exportar CSV actualizado

#### **Dependencias entre variables agregadas**

- **Independientes**: las variables agregadas 1, 2, 3 y 6 se pueden crear en paralelo.
- **Variable agregada 4**: depende de las variables 1, 2 y 3.
- **Variable agregada 5**: depende de la variable 4
- **Variable agregada 7**: depende de la variable 1

#### **Hipótesis que validan**

1. La variable `nomad_housing_cost` está relacionada con la verificación de las hipótesis 1 y 4 del planteamiento del EDA. 
2. La variable `basic_basket_index` con las hipótesis 1 y 2. 
3. La variable `daily_meal_cost` con la hipótesis 1. 
4. La variable `monthly_nomad_cost` con las hipótesis 1 y 5. 
5. La variable `local_purchasing_power` con la hipótesis 2. 
6. La variable `cappuccino_index` con la hipótesis 1. 
7. La variable `housing_salary_ratio` con las hipótesis 2 y 4. 

### 1. **Importar librerías y cargar el dataset de Cost of Living**

In [48]:
import numpy as np
import pandas as pd

pd.options.mode.copy_on_write = True # CoW por defecto a partir de pandas 3.0.0 

In [49]:
df_cost = pd.read_csv("./data/cost-of-living-clean.csv")
df_cost.head(20) # Cargamos el CSV limpios de Cost of Living y visualizamos 20 filas y tenerlo cargado

,city_name,country_name,meal_inexpensive_restaurant,meal_midrange_restaurant_2p,mcmeal_fastfood,beer_domestic_restaurant_0_5l,beer_imported_restaurant_0_33l,cappuccino_restaurant,soda_restaurant_0_33l,water_restaurant_0_33l,...,rent_1br_city_center,rent_1br_outside_center,rent_3br_city_center,rent_3br_outside_center,price_sqm_city_center,price_sqm_outside_center,avg_net_salary,mortgage_interest_rate_20y,data_quality_flag,continent
0,Seoul,South Korea,7.68,53.78,6.15,3.07,4.99,3.93,1.48,0.79,...,742.54,557.52,2669.12,1731.08,22067.70,10971.90,2689.62,3.47,1,Asia
1,Shanghai,China,5.69,39.86,5.69,1.14,4.27,3.98,0.53,0.33,...,1091.93,569.88,2952.70,1561.59,17746.11,9416.35,1419.87,5.03,1,Asia
2,Guangzhou,China,4.13,28.47,4.98,0.85,1.71,3.54,0.44,0.33,...,533.28,317.45,1242.24,688.05,12892.82,5427.45,1211.68,5.19,1,Asia
3,Mumbai,India,3.68,18.42,3.68,2.46,4.30,2.48,0.48,0.19,...,522.40,294.05,1411.12,699.80,6092.45,2777.51,640.81,7.96,1,Asia
4,Delhi,India,4.91,22.11,4.30,1.84,3.68,1.77,0.49,0.19,...,229.84,135.31,601.02,329.15,2506.73,1036.74,586.46,8.06,1,Asia
5,Dhaka,Bangladesh,1.95,11.71,4.88,5.85,5.12,1.95,0.29,0.16,...,142.09,87.79,347.57,208.50,1119.98,571.72,280.73,9.26,1,Asia
6,Osaka,Japan,7.45,48.39,5.36,3.35,3.72,3.28,1.09,0.81,...,674.96,376.14,1737.21,993.17,8043.38,4825.58,2322.46,1.49,1,Asia
7,Jakarta,Indonesia,2.59,22.69,3.57,2.06,3.24,2.23,0.61,0.27,...,505.59,277.43,1172.14,615.04,2632.80,1241.09,509.12,9.05,1,Asia
8,Shenzhen,China,4.27,28.47,4.98,1.14,3.99,4.20,0.47,0.34,...,738.75,435.07,1682.30,886.16,17898.73,8091.57,1572.22,4.99,1,Asia
9,Kinshasa,Congo,15.11,42.63,10.08,1.74,2.50,4.35,2.78,0.84,...,2000.00,725.00,4500.00,1160.00,6170.63,933.33,400.00,19.33,0,Africa


#### **2. Calculamos la variable agregada 1: `nomad_housing_cost`**

Coste medio de alquiler para un nómada digital (1 dormitorio).

¿Por qué 1 dormitorio y precios relacionados con el alquiler? 

En general los nómadas digital prefieren alquilar y no comprar por movilidad constante y suelen hacerlo en etapas de su vida sin familia.

En este sentido, no usamos las variables de alquiler de pisos de 3 dormitorios y el coste de comprar una vivienda. 

**La variable agregada sería sima de los costes de alquiler de vivienda de 1 dormitorio en el centro de la ciudad y a la afueras**: 

**Cálculo variable agregada**: `(rent_1br_city_center + rent_1br_outside_center) / 2`

In [52]:
# Promedio entre alquiler en centro y afueras (1 dormitorio)
# Representa el coste típico de vivienda para un nómada digital

df_cost['nomad_housing_cost'] = (
    df_cost['rent_1br_city_center'] + 
    df_cost['rent_1br_outside_center']
) / 2

# Valores estadísticos estándar de la nueva variable que usaremos en el EDA 

print(f"Valores estadísticos de la variable agregada relacionada con alquiler de vivienda de 1 dormitorio")
print(f"-------------------------------------------------------------------------------------------------")
print(df_cost['nomad_housing_cost'].describe())

Valores estadísticos de la variable agregada relacionada con alquiler de vivienda de 1 dormitorio
-------------------------------------------------------------------------------------------------
count     4742.000000
mean       636.678457
std        553.182086
min         18.985000
25%        219.809375
50%        471.722500
75%        945.632500
max      10799.100000
Name: nomad_housing_cost, dtype: float64


#### **3. Calculamos la variable agregada 2: `basic_basket_index`**

Coste promedio productos básicos supermercado;

¿Por qué es importante tener en cuenta el precio de una canasta básica?

El costo de los productos básicos es uno de los gastos principales para las personas nómadas digitales. Analizar esta variable permite comparar ciudades y estimar dónde es más accesible cubrir necesidades esenciales de alimentación, identificando lugares con canastas básicas más completas y a precios más asequibles.


In [53]:
# Identificamos que columnas "parecen" de supermercado 

cols = df_cost.columns

grocery_candidates = [c for c in cols if any(k in c.lower() for k in [
    "milk", "bread", "rice", "eggs", "cheese", "chicken", "beef",
    "apples", "banana", "oranges", "tomato", "potato", "onion",
    "lettuce", "water", "supermarket", "market"
])]

grocery_candidates

['water_restaurant_0_33l',
 'milk_1l',
 'bread_white_500g',
 'rice_white_1kg',
 'eggs_12',
 'cheese_local_1kg',
 'chicken_fillet_1kg',
 'beef_1kg',
 'apples_1kg',
 'bananas_1kg',
 'oranges_1kg',
 'tomatoes_1kg',
 'potatoes_1kg',
 'onions_1kg',
 'lettuce_1unit',
 'water_1_5l_supermarket',
 'wine_midrange_supermarket',
 'beer_domestic_supermarket_0_5l',
 'beer_imported_supermarket_0_33l',
 'price_sqm_city_center',
 'price_sqm_outside_center']

In [54]:
# Filtro para quedarse solo con opciones de supermercado ya que aparece "water_restaurant", "price_sqm_city_center"
# y "price_sqm_outside_center", no aplicables como canasta basica

grocery_cols = [
    c for c in grocery_candidates
    if("restaurant" not in c.lower())
    and("water_restaurant" not in c.lower())
    and("price_sqm" not in c.lower())
]
grocery_cols

['milk_1l',
 'bread_white_500g',
 'rice_white_1kg',
 'eggs_12',
 'cheese_local_1kg',
 'chicken_fillet_1kg',
 'beef_1kg',
 'apples_1kg',
 'bananas_1kg',
 'oranges_1kg',
 'tomatoes_1kg',
 'potatoes_1kg',
 'onions_1kg',
 'lettuce_1unit',
 'water_1_5l_supermarket',
 'wine_midrange_supermarket',
 'beer_domestic_supermarket_0_5l',
 'beer_imported_supermarket_0_33l']

In [55]:
# Visualizamos 20 filas para tenerlo cargado
 
grocery_cols = [
    "milk_1l","bread_white_500g","rice_white_1kg","eggs_12","cheese_local_1kg",
    "chicken_fillet_1kg","beef_1kg","apples_1kg","bananas_1kg","oranges_1kg",
    "tomatoes_1kg","potatoes_1kg","onions_1kg"
]

df_cost[["city_name", "country_name"] + grocery_cols].head(20)

,city_name,country_name,milk_1l,bread_white_500g,rice_white_1kg,eggs_12,cheese_local_1kg,chicken_fillet_1kg,beef_1kg,apples_1kg,bananas_1kg,oranges_1kg,tomatoes_1kg,potatoes_1kg,onions_1kg
0,Seoul,South Korea,2.20,2.85,3.53,4.04,11.54,10.58,41.61,6.77,3.71,6.50,6.19,3.84,2.92
1,Shanghai,China,2.74,2.61,1.22,2.22,18.35,4.86,13.12,2.26,1.60,2.19,1.53,0.84,1.04
2,Guangzhou,China,1.91,1.63,1.03,1.71,9.00,3.77,11.75,2.02,1.44,1.82,1.31,0.74,1.00
3,Mumbai,India,0.75,0.50,0.83,0.95,5.88,3.69,5.95,2.09,0.67,1.34,0.59,0.44,0.44
4,Delhi,India,0.73,0.50,0.85,1.02,4.36,3.81,5.71,1.79,0.75,1.03,0.61,0.37,0.41
5,Dhaka,Bangladesh,0.83,0.67,0.69,1.32,7.21,3.07,7.19,2.38,1.04,2.18,0.96,0.28,0.49
6,Osaka,Japan,1.41,1.47,4.92,1.90,11.09,6.93,22.46,4.09,1.85,3.87,6.34,3.31,2.01
7,Jakarta,Indonesia,1.30,1.20,0.83,1.71,6.90,3.52,8.49,2.99,1.60,2.13,1.30,1.42,2.05
8,Shenzhen,China,2.23,2.40,1.00,2.13,13.67,4.37,15.56,1.97,1.47,1.81,1.45,1.06,1.16
9,Kinshasa,Congo,2.00,1.33,5.17,4.15,9.50,5.00,20.00,10.00,3.25,5.25,6.33,3.60,2.83


In [57]:
df_cost["basic_basket_total"] = df_cost[grocery_cols].sum(axis=1)

In [58]:
# Aqui empezamos a ordenar de menor a mayor para identificar las mas asequibles y de mayor a menor para saber que tanta 
# direncia hay y deducir si hay precios muy extremos que puedan distorcionar el indice, 
# entonces se trabaja con ¿medias o medianas?: 
# De menor a mayor

df_cost[["city_name", "country_name", "basic_basket_total"] + grocery_cols] \
    .sort_values(by="basic_basket_total", ascending=True) \
    .head(20)

,city_name,country_name,basic_basket_total,milk_1l,bread_white_500g,rice_white_1kg,eggs_12,cheese_local_1kg,chicken_fillet_1kg,beef_1kg,apples_1kg,bananas_1kg,oranges_1kg,tomatoes_1kg,potatoes_1kg,onions_1kg
3571,Toba Tek Singh,Pakistan,11.580,0.36,0.99,0.72,0.800,1.79,1.34,2.24,0.89,0.67,0.89,0.45,0.13,0.31
1448,Sahiwal,Pakistan,11.980,0.47,0.45,0.79,0.920,2.74,1.68,2.46,0.86,0.46,0.22,0.45,0.27,0.21
3261,Krishnapur,India,12.130,0.49,0.18,0.61,0.830,2.46,1.84,1.84,1.84,0.31,0.74,0.37,0.31,0.31
648,Bahawalpur,Pakistan,12.240,0.48,0.35,0.69,0.820,2.84,1.71,2.68,0.80,0.44,0.48,0.44,0.25,0.26
918,Shekhupura,Pakistan,12.270,0.45,0.46,0.82,0.920,2.74,1.68,2.46,0.86,0.46,0.22,0.72,0.27,0.21
1244,Mardan,Pakistan,12.440,0.45,0.43,0.63,0.890,3.80,1.34,2.41,0.60,0.36,0.42,0.63,0.26,0.22
1035,Dera Ghazi Khan,Pakistan,12.490,0.53,0.35,0.69,0.820,2.84,1.71,2.79,0.89,0.44,0.58,0.36,0.22,0.27
914,Chiniot,Pakistan,12.500,0.38,0.24,0.51,0.910,3.80,1.70,2.68,0.49,0.31,0.54,0.54,0.18,0.22
1006,Jhang City,Pakistan,12.630,0.47,0.27,0.79,0.780,3.26,1.71,2.68,0.80,0.52,0.48,0.36,0.25,0.26
150,Faisalabad,Pakistan,12.700,0.53,0.35,0.69,0.820,2.84,1.71,2.79,0.89,0.44,0.58,0.44,0.34,0.28


In [59]:
#De mayor a menor y comprobamos que los precios cambian considerablemente:

df_cost[["city_name", "country_name", "basic_basket_total"] + grocery_cols] \
    .sort_values(by="basic_basket_total", ascending=False) \
    .head(20)

,city_name,country_name,basic_basket_total,milk_1l,bread_white_500g,rice_white_1kg,eggs_12,cheese_local_1kg,chicken_fillet_1kg,beef_1kg,apples_1kg,bananas_1kg,oranges_1kg,tomatoes_1kg,potatoes_1kg,onions_1kg
3128,Aarau,Switzerland,197.08,1.89,3.05,2.19,5.64,35.97,33.12,96.16,4.06,2.56,5.29,1.98,2.78,2.39
3215,Schwyz,Switzerland,197.00,1.66,3.18,4.27,6.24,37.40,26.71,90.82,3.74,3.74,4.27,8.39,3.37,3.21
432,Monrovia,Liberia,171.12,1.40,1.43,4.71,5.33,15.22,9.87,41.52,22.74,13.78,11.62,15.41,15.87,12.22
3272,Greenville,Liberia,169.57,1.40,1.43,4.71,5.33,13.67,9.87,41.52,22.74,13.78,11.62,15.41,15.87,12.22
4562,Affoltern am Albis,Switzerland,161.79,1.92,1.82,1.82,7.69,26.71,37.40,64.11,5.34,3.21,3.21,3.21,3.21,2.14
1744,Basel,Switzerland,160.45,1.71,3.49,2.77,5.88,34.28,24.39,66.85,4.28,2.89,4.19,4.27,2.82,2.63
3066,Dietikon,Switzerland,156.85,1.71,2.96,2.88,7.27,31.52,29.20,57.70,4.74,3.13,4.91,4.27,3.21,3.35
2580,Lucerne,Switzerland,155.42,1.71,3.69,2.27,6.34,22.44,26.71,73.01,3.50,3.05,3.39,3.54,2.99,2.78
4518,Meilen,Switzerland,154.76,1.98,4.27,4.27,5.13,26.71,41.67,53.42,4.06,2.14,3.74,4.06,2.03,1.28
3064,Dubendorf,Switzerland,151.94,1.71,3.45,2.27,6.34,22.44,26.71,66.96,3.50,3.05,3.39,5.56,3.21,3.35


In [60]:
# Canasta básica: media (índice), mediana (robusta) y total (costo de la canasta).

df_cost["basic_basket_index_mean"] = df_cost[grocery_cols].mean(axis=1)
df_cost["basic_basket_index_median"] = df_cost[grocery_cols].median(axis=1)
df_cost["basic_basket_total"] = df_cost[grocery_cols].sum(axis=1)

df_cost[["basic_basket_index_mean","basic_basket_index_median","basic_basket_total"]].describe()

,basic_basket_index_mean,basic_basket_index_median,basic_basket_total
count,4742.000000,4742.000000,4742.000000
mean,3.721408,2.214423,48.378307
std,1.677218,1.228781,21.803832
min,0.890769,0.390000,11.580000
25%,2.402500,1.300000,31.232500
50%,3.433846,1.890000,44.640000
75%,4.804038,2.970000,62.452500
max,15.160000,12.220000,197.080000


---Aquí podriamos empezar a hacer comparativas de que paises son mas costosos o asequibles pero todo depende del
sueldo de un nomada, ya que la pregunta es: ¿Hay nomadas con contratos Estadounidenses trabajando desde la India, Nepal,
Colombia, México? Etc

O Nomadas con contratos Españoles viviendo en Suiza, Estados Unidos, Austria, etc?

¿Que sugiririamos a un Nómada segun a datos los paises mas asequibles y con una calidad de vida balanceada? 

#### **4. Calculamos la variable agregada 3 : `daily_meal_cost`**

Cappuccino + comida restaurante económico

Costo estimado de una comida intermitente fuera de casa, calculado como la suma del precio de una comida en restaurante económico y un cappuccino. Esta variable aproxima un gasto típico en días en los que una persona nómada digital no puede cocinar (por falta de tiempo, alojamiento sin cocina, trabajo en coworking o necesidad de comer fuera mientras se mueve) y permite comparar ciudades en términos de accesibilidad para cubrir una comida básica fuera de casa.

**Cálculo variable agregada**: `meal_inexpensive_restaurant + cappuccino_restaurant`

In [61]:
df_cost['daily_meal_cost'] = (
    df_cost['meal_inexpensive_restaurant'] + 
    df_cost['cappuccino_restaurant']
)

df_cost["daily_meal_cost"].describe()

count    4742.000000
mean       13.135526
std         7.959227
min         1.100000
25%         6.372500
50%        12.255000
75%        18.920000
max        64.280000
Name: daily_meal_cost, dtype: float64

In [62]:
#Para tener mas claro añado un top 10 de ciudades mas caras y considerando que es un costo por día

df_cost[["city_name","country_name","daily_meal_cost"]].sort_values("daily_meal_cost", ascending=False).head(10)

,city_name,country_name,daily_meal_cost
1019,Turkmenabat,Turkmenistan,64.28
1525,Dasoguz,Turkmenistan,62.85
1785,Lorain,United States,58.25
3771,Lake Havasu City,United States,56.50
2991,South Miami Heights,United States,56.00
3266,Summit,United States,55.50
4328,Cranford,United States,54.50
3807,Wayne,United States,54.00
4658,Mechanicsburg,United States,53.83
4695,Lihue,United States,50.00


In [63]:
# Y un top 10 de los mas asequibles

df_cost[["city_name","country_name","daily_meal_cost"]].sort_values("daily_meal_cost", ascending=True).head(10)

,city_name,country_name,daily_meal_cost
918,Shekhupura,Pakistan,1.10
718,Sargodha,Pakistan,1.12
1244,Mardan,Pakistan,1.12
621,Akure,Nigeria,1.13
1381,Djelfa,Algeria,1.20
1635,Mandi Burewala,Pakistan,1.20
2335,Jendouba,Tunisia,1.29
2129,Laghouat,Algeria,1.30
2368,Gafsa,Tunisia,1.30
2261,Ghardaia,Algeria,1.33


In [64]:
# Aqui le pedi a chat me filtrara por rango intermedio 40, 60, ¿ Y por que ese rango?

# Porque es una forma práctica de quedarse con el “centro” de la distribución y evitar tanto los valores muy bajos 
# como los muy altos.

# Percentil 40 = el valor por debajo del cual está el 40% de las ciudades (más baratas).

# Percentil 60 = el valor por debajo del cual está el 60% de las ciudades.

# Entonces, entre 40 y 60 te quedas con el 20% más “intermedio” (las ciudades alrededor de la mediana),
#  que suele representar “ni caro ni barato”.

low = df_cost["daily_meal_cost"].quantile(0.40)
high = df_cost["daily_meal_cost"].quantile(0.60)

df_cost.loc[
    df_cost["daily_meal_cost"].between(low, high),
    ["city_name","country_name","daily_meal_cost"]
].head(20)

,city_name,country_name,daily_meal_cost
0,Seoul,South Korea,11.61
1,Shanghai,China,9.67
6,Osaka,Japan,10.73
14,Mexico City,Mexico,10.45
19,Tokyo,Japan,10.87
33,Nagoya,Japan,10.10
43,Luanda,Angola,11.77
62,Hong Kong,Hong Kong,12.79
66,Santiago,Chile,11.67
70,Riyadh,Saudi Arabia,10.49


Entonces considerando los 3 rangos de precios, se podria ahora visualizar la calidad de vida dentro de esos top 10.

#### **4. Calculamos la variable agregada 4 : `monthly_nomad_cost`**

Coste mensual total estimado para un nómada digital.

La variable agregada se calcula de una proyección mensual del coste de las siguientes variables:

Vivienda: nomad_housing_cost (la variable agregada 1)

Alimentación: (daily_meal_cost × 8) + (basic_basket_index × 30) (variables agregadas 2 y 3 pero con proyecciones de coste diferentes: 30% días

comer fuera 2 veces por semana y 100% comer en casa)

Internet: internet_60mbps_unlimited

Utilities: utilities_85sqm

Transporte: public_transport_monthly_pass


In [65]:
# Obtenemos un resumen estadístico rápido sobe transporte publico 

df_cost["public_transport_monthly_pass"].describe()

count    4742.000000
mean       40.789237
std        31.447121
min         0.000000
25%        17.790000
50%        36.880000
75%        50.000000
max       369.860000
Name: public_transport_monthly_pass, dtype: float64

In [66]:
# Aplicamos la misma logica de la anterior variable y buscamos un top 10 de transporte publico mas caro

df_cost[["city_name","country_name","public_transport_monthly_pass"]].sort_values("public_transport_monthly_pass", ascending=False).head(10)

,city_name,country_name,public_transport_monthly_pass
2688,Zaandam,Netherlands,369.86
2767,Royal Tunbridge Wells,United Kingdom,320.42
2984,Ali Sabieh,Djibouti,308.99
477,Djibouti,Djibouti,308.99
3148,Massapequa,United States,300.00
3875,Welwyn Garden City,United Kingdom,276.75
4625,Lachen,Switzerland,267.11
4333,Loon op Zand,Netherlands,263.44
4636,River Grove,United States,250.00
4359,Braidwood,United States,250.00


In [67]:
# Y un top 10 de los mas asequibles
# Y aqui obtenemos 72 valores a 0.0 lo cual ignoraremos 

df_cost.loc[df_cost["public_transport_monthly_pass"] > 0,
            ["city_name","country_name","public_transport_monthly_pass"]] \
  .sort_values("public_transport_monthly_pass", ascending=True) \
  .head(10)

,city_name,country_name,public_transport_monthly_pass
2494,Galle,Sri Lanka,0.81
948,Tanta,Egypt,1.02
2463,Erdenet,Mongolia,1.47
1601,Bila Tserkva,Ukraine,1.63
440,Quetta,Pakistan,1.68
681,Warangal,India,2.21
4660,Gilgit,Pakistan,2.24
519,Sri Jayewardenepura Kotte,Sri Lanka,2.44
2925,Matale,Sri Lanka,2.44
607,Jalandhar,India,2.46


In [68]:
#Habria que buscarles la media? 

(df_cost["public_transport_monthly_pass"] == 0).sum()

np.int64(72)

In [79]:
#Seguimos la misma logica y hacemos una mediana, aunque teniendo valores a 0 podria no afectar tanto ya que de 4742 filas, solo 67 no tienen 
#valor y aqui habria de definir si darles una media o eliminarlas

# Percentil 40 = el valor por debajo del cual está el 40% de las ciudades (más baratas).

# Percentil 60 = el valor por debajo del cual está el 60% de las ciudades.

# Entonces, entre 40 y 60 te quedas con el 20% más “intermedio” (las ciudades alrededor de la mediana),
#  que suele representar “ni caro ni barato”.

low_price_transport = df_cost["public_transport_monthly_pass"].quantile(0.40)
high_price_transport = df_cost["public_transport_monthly_pass"].quantile(0.60)

df_cost.loc[
    df_cost["public_transport_monthly_pass"].between(low_price_transport, high_price_transport),
    ["city_name","country_name","public_transport_monthly_pass"]
].head(20)

,city_name,country_name,public_transport_monthly_pass
0,Seoul,South Korea,42.25
1,Shanghai,China,28.47
2,Guangzhou,China,28.47
8,Shenzhen,China,35.59
9,Kinshasa,Congo,30.00
10,Bangkok,Thailand,37.44
15,Lagos,Nigeria,33.77
17,Beijing,China,42.71
18,Moscow,Russia,38.40
22,Istanbul,Turkey,32.31


In [70]:
# Utilizamos el codigo de Juan modificando un poco y después concluimos
# Coste mensual total para un nómada digital, que incluye: 
# 1. Vivienda. 
# 2. Alimentación.
# 3. Internet.
# 4. Suministros tipo gas, luz...
# 5. Transporte.

# Calculamos el coste mensual de alimentación.  
# 8 días comiendo fuera y 30 días consumiendo la compra mensual en el supermercado)

monthly_food_cost = (
    df_cost['daily_meal_cost'] * 8 +       
    df_cost['basic_basket_total'] * 30     
)

# Calculamos el coste mensual total con todas las variables agregadas y no agregadas. 

df_cost['monthly_nomad_cost'] = (
    df_cost['nomad_housing_cost'] +        
    monthly_food_cost +                      
    df_cost['internet_60mbps_unlimited'] +  
    df_cost['utilities_85sqm'] +            
    df_cost['public_transport_monthly_pass'] 
)

# Datos estadísticos generales de la nueva variable agregada 

print(f"Valores estadísticos de la variable agregada relacionada con el coste mensual total:")
print(f"------------------------------------------------------------------------------------")
print(df_cost['monthly_nomad_cost'].describe())

Valores estadísticos de la variable agregada relacionada con el coste mensual total:
------------------------------------------------------------------------------------
count      4742.000000
mean       2512.269918
std        7171.961991
min         472.365000
25%        1370.881250
50%        2245.046250
75%        3301.265000
max      488963.400000
Name: monthly_nomad_cost, dtype: float64


#### **6. Calculamos la variable agregada 5: `local_purchasing_power`**

Poder adquisitivo local relativo al coste de vida nómada.

**Básicamente la variable calcula el promedio del salario neto dividio el coste mensual total: `avg_net_salary / monthly_nomad_cost`**

Lo interesante aquí es saber en cuántas ciudades el salario está por encima de coste o no.

In [72]:
# Cálculo de la relación entre salario neto y coste de vida mensual:

df_cost['local_purchasing_power'] = (
    df_cost['avg_net_salary'] / df_cost['monthly_nomad_cost']
)

# Datos estadísticos generales de la nueva variable agregada: capacidad de compra

print(f"Valores estadísticos de la variable agregada relacionada con capacidad de compra:")
print(f"------------------------------------------------------------------------------------")
print(df_cost['local_purchasing_power'].describe())

# Y calculamos números de ciudades donde el salario está por debajo del coste
# Hay casi 1300 ciudades donde el salario neto promedio no permite subsistir 

print(f"------------------------------------------------------------------------------------")
print(f"Ciudades donde salario > coste: {(df_cost['local_purchasing_power'] > 1).sum()}")
print(f"Ciudades donde salario < coste: {(df_cost['local_purchasing_power'] < 1).sum()}")

Valores estadísticos de la variable agregada relacionada con capacidad de compra:
------------------------------------------------------------------------------------
count    4742.000000
mean        0.669040
std         0.378225
min         0.001470
25%         0.356223
50%         0.614196
75%         0.934273
max         3.032523
Name: local_purchasing_power, dtype: float64
------------------------------------------------------------------------------------
Ciudades donde salario > coste: 921
Ciudades donde salario < coste: 3821


 #### **7. Calculamos la variable agregada 6: `cappuccino_index`**

El precio del cappuccino varía mucho entre ciudades y puede tener rangos muy distintos. Para poder comparar de forma justa, normalizamos esta
 variable transformando el precio original a un índice en una escala común. Así, el cappuccino_index representa el costo relativo del cappuccino
  en cada ciudad (más alto = más caro, más bajo = más barato), facilitando comparaciones y visualizaciones sin que el análisis se vea dominado
  por la magnitud del precio.
  

In [73]:
#Normalización Min-Max (escala 0 a 1)

#Interpretación: 0 = ciudad más barata, 1 = ciudad más cara.

min_val = df_cost["cappuccino_restaurant"].min()
max_val = df_cost["cappuccino_restaurant"].max()

df_cost["cappuccino_index"] = (df_cost["cappuccino_restaurant"] - min_val) / (max_val - min_val)
df_cost["cappuccino_index"].describe()

count    4742.000000
mean        0.258099
std         0.138229
min         0.000000
25%         0.139059
50%         0.245399
75%         0.360941
max         1.000000
Name: cappuccino_index, dtype: float64

In [ ]:
#Aqui volvemos a hacer un top 10 de mas asequibles

df_cost[["city_name","country_name","cappuccino_restaurant","cappuccino_index"]] \
  .sort_values("cappuccino_index", ascending=True).head(10)

,city_name,country_name,cappuccino_restaurant,cappuccino_index
2129,Laghouat,Algeria,0.22,0.000000
2081,Medea,Algeria,0.22,0.000000
1845,Bordj Bou Arreridj,Algeria,0.24,0.002045
2014,Mostaganem,Algeria,0.25,0.003067
2867,Puttalam,Sri Lanka,0.27,0.005112
1614,Moratuwa,Sri Lanka,0.27,0.005112
2858,Ratnapura,Sri Lanka,0.27,0.005112
2137,Batticaloa,Sri Lanka,0.27,0.005112
2568,El Bayadh,Algeria,0.29,0.007157
1381,Djelfa,Algeria,0.30,0.008180


In [ ]:
# Top 10 mas caras 

df_cost[["city_name","country_name","cappuccino_restaurant","cappuccino_index"]] \
  .sort_values("cappuccino_index", ascending=False).head(10)

,city_name,country_name,cappuccino_restaurant,cappuccino_index
4355,Gallup,United States,10.00,1.000000
4375,Perrysburg,United States,8.00,0.795501
3291,Plainedge,United States,8.00,0.795501
4677,Menominee,United States,8.00,0.795501
2016,Topeka,United States,7.67,0.761759
1019,Turkmenabat,Turkmenistan,7.14,0.707566
2612,Cheyenne,United States,7.03,0.696319
4541,Big Rapids,United States,7.00,0.693252
4016,Spanish Fork,United States,7.00,0.693252
2405,League City,United States,7.00,0.693252


In [78]:
# Y seguimos aplicando la misma logica

low_price_cappuccino = df_cost["cappuccino_restaurant"].quantile(0.40)
high_price_capuccino = df_cost["cappuccino_restaurant"].quantile(0.60)

df_cost.loc[
    df_cost["cappuccino_restaurant"].between(low_price_cappuccino, high_price_capuccino),
    ["city_name","country_name","cappuccino_restaurant"]
].head(20)

,city_name,country_name,cappuccino_restaurant
3,Mumbai,India,2.48
7,Jakarta,Indonesia,2.23
10,Bangkok,Thailand,2.13
14,Mexico City,Mexico,2.71
20,Manila,Philippines,2.97
29,Wuhan,China,2.86
33,Nagoya,Japan,2.65
36,Baoding,China,2.85
37,Lima,Peru,2.66
40,Nanyang,China,2.73


#### **8. Calculamos la variable agregada 7: `housing_salary_ratio`**